In [1]:
# 1. Data Loading and Inspection

import pandas as pd
from sklearn.datasets import load_breast_cancer

# Load dataset
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Display first 5 rows
print("First 5 rows:")
display(df.head())


First 5 rows:


mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0                 0.07871  ...          17.33           184.60      2019.0   
1                 0.05667  ...          23.41           158.80      1956.0   
2                 0.05999  ...          25.53           152.50      1709.0   
3                 0.09744  ...          26.50            98.87       567.7   
4                 0.05883  ...          16.67           152.20      1575.0   

   worst smoothness  worst compactness  worst concavity  worst concave points  \
0            0.1622             0.6656           0.7119                0.2654   
1            0.1238             0.1866           0.2416                0.1860   
2            0.1444             0.4245           0.4504                0.2430   
3            0.2098             0.8663           0.6869                0.2575   
4            0.1374             0.2050           0.4000                0.1625   

   worst symmetry  worst fractal dimension  target  
0          0.4601                  0.11890       0  
1          0.2750                  0.08902       0  
2          0.3613                  0.08758       0  
3          0.6638                  0.17300       0  
4          0.2364                  0.07678       0  

[5 rows x 31 columns]

In [2]:

# Print class counts
print("\nClass distribution:")
print(df['target'].value_counts())



Class distribution:
target
1    357
0    212
Name: count, dtype: int64


In [3]:
# 2. Data Splitting
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']

# Stratified split to preserve class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape:     {X_test.shape}")
print(f"Train class distribution:\n{y_train.value_counts()}")
print(f"Test class distribution:\n{y_test.value_counts()}")


Training data shape: (398, 30)
Test data shape:     (171, 30)
Train class distribution:
target
1    250
0    148
Name: count, dtype: int64
Test class distribution:
target
1    107
0     64
Name: count, dtype: int64


In [4]:
# 3. Initial Model Training

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, recall_score, accuracy_score

model = LogisticRegression(max_iter=10000, random_state=42)  # large max_iter to ensure convergence
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

In [5]:
# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {acc:.4f}")

# Recall (for each class)
recall_0 = recall_score(y_test, y_pred, pos_label=0)
recall_1 = recall_score(y_test, y_pred, pos_label=1)
print(f"Recall for class 0 (benign): {recall_0:.4f}")
print(f"Recall for class 1 (malignant): {recall_1:.4f}")

# Full classification report
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=['Benign (0)', 'Malignant (1)']))


Accuracy: 0.9474
Recall for class 0 (benign): 0.8906
Recall for class 1 (malignant): 0.9813

Classification Report:

               precision    recall  f1-score   support

   Benign (0)       0.97      0.89      0.93        64
Malignant (1)       0.94      0.98      0.96       107

     accuracy                           0.95       171
    macro avg       0.95      0.94      0.94       171
 weighted avg       0.95      0.95      0.95       171



In [7]:
# 4. Feature Importance Extraction
import numpy as np

# Get absolute coefficients
coef_abs = np.abs(model.coef_[0])

# Create dataframe of features and their importance
feature_importance_df = pd.DataFrame({
    "feature": data.feature_names,
    "coefficient_abs": coef_abs
})

# Sort descending
feature_importance_df = feature_importance_df.sort_values(by="coefficient_abs", ascending=False)

print("Top 5 most influential features:")
display(feature_importance_df.head(5))


Top 5 most influential features:


feature  coefficient_abs
26    worst concavity         1.258799
0         mean radius         1.069444
25  worst compactness         0.843785
11      texture error         0.836586
28     worst symmetry         0.743533

In [8]:
# 5. Reduced Feature Model (using top 5 features)

top_5_features = feature_importance_df.head(5)['feature'].tolist()

X_train_top5 = X_train[top_5_features]
X_test_top5 = X_test[top_5_features]

model_top5 = LogisticRegression(max_iter=10000, random_state=42)
model_top5.fit(X_train_top5, y_train)

y_pred_top5 = model_top5.predict(X_test_top5)

In [9]:
acc_top5 = accuracy_score(y_test, y_pred_top5)
recall_0_top5 = recall_score(y_test, y_pred_top5, pos_label=0)
recall_1_top5 = recall_score(y_test, y_pred_top5, pos_label=1)

print(f"Accuracy with top 5 features: {acc_top5:.4f}")
print(f"Recall for class 0 with top 5 features: {recall_0_top5:.4f}")
print(f"Recall for class 1 with top 5 features: {recall_1_top5:.4f}")

Accuracy with top 5 features: 0.9123
Recall for class 0 with top 5 features: 0.8750
Recall for class 1 with top 5 features: 0.9346


In [10]:
#===============================================#
#-------------------Additional------------------#
#===============================================#
# Manual Hyperparameter Tuning on full feature set

C_values = [0.01, 0.1, 1, 10, 100]
results = []

for C in C_values:
    model_tuned = LogisticRegression(C=C, max_iter=10000, random_state=42)
    model_tuned.fit(X_train, y_train)
    y_pred_c = model_tuned.predict(X_test)
    acc_c = accuracy_score(y_test, y_pred_c)
    recall_0_c = recall_score(y_test, y_pred_c, pos_label=0)
    recall_1_c = recall_score(y_test, y_pred_c, pos_label=1)
    results.append((C, acc_c, recall_0_c, recall_1_c))

# Create results DataFrame
tuning_df = pd.DataFrame(results, columns=['C', 'Accuracy', 'Recall_0', 'Recall_1'])
print(tuning_df)


        C  Accuracy  Recall_0  Recall_1
0    0.01  0.947368  0.906250  0.971963
1    0.10  0.959064  0.921875  0.981308
2    1.00  0.947368  0.890625  0.981308
3   10.00  0.941520  0.875000  0.981308
4  100.00  0.953216  0.890625  0.990654
